In [11]:
import os
import pandas as pd
import folium
from geopy.distance import geodesic
from folium.features import DivIcon

In [3]:
data_processed_path = os.path.join('..','data','processed')
stations_lithuania = pd.read_csv(os.path.join(data_processed_path, 'lithuania_coordinates_cleaned.csv'))

In [5]:
csv_file_path = '../data/processed/lithuania_coordinates_cleaned.csv'

lat_col = 'latitude'
lon_col = 'longitude'
city_col = 'name'

map_center = [55.17, 23.88] 
zoom_start = 7
output_map_file = '../plots/lithuania_map_from_csv.html'

try:
    df = pd.read_csv(csv_file_path)
    print(f"  {len(df)} -- num of points from {csv_file_path}.")
except FileNotFoundError:
    print(f"Error: file '{csv_file_path}' didnt exist...")
    exit() 

lithuania_map = folium.Map(location=map_center, zoom_start=zoom_start)

for index, row in df.iterrows():
    lat = row[lat_col]
    lon = row[lon_col]
    city_name = row[city_col]

    popup_text = f"City: {city_name}<br>Coordinates: {lat:.4f}, {lon:.4f}"

    folium.Marker(
        location=[lat, lon],
        popup=popup_text,
        tooltip=city_name 
    ).add_to(lithuania_map)

lithuania_map.save(output_map_file)

print(f"Map created at '{output_map_file}'. Open in browser:)")

  52 -- num of points from ../data/processed/lithuania_coordinates_cleaned.csv.
Map created at '../notes/plots/lithuania_map_from_csv.html'. Open in browser:)


In [6]:
# --- Configuration ---
csv_file_path = '../data/processed/lithuania_coordinates_cleaned.csv'
lat_col = 'latitude'
lon_col = 'longitude'
city_col = 'name'

# Map settings
map_center = [55.17, 23.88] 
zoom_start = 7
output_map_file = '../plots/lithuania_map_with_distances.html'

# --- 1. Load Data ---
try:
    df = pd.read_csv(csv_file_path)
    print(f"Successfully loaded {len(df)} points from '{csv_file_path}'.")
except FileNotFoundError:
    print(f"Error: The file '{csv_file_path}' was not found.")
    exit()

# --- 2. Create Base Map ---
lithuania_map = folium.Map(location=map_center, zoom_start=zoom_start)

# --- 3. Add City Markers ---
# This loop adds a marker for each city, just like in your original script.
for index, row in df.iterrows():
    lat = row[lat_col]
    lon = row[lon_col]
    city_name = row[city_col]
    popup_text = f"City: {city_name}<br>Coordinates: {lat:.4f}, {lon:.4f}"

    folium.Marker(
        location=[lat, lon],
        popup=popup_text,
        tooltip=city_name 
    ).add_to(lithuania_map)

# --- 4. Add Lines and Distance Labels ---
# This new section connects consecutive points with a line and adds a distance label.
# We loop to the second-to-last point to avoid an index error on the last item.
for i in range(len(df) - 1):
    # Get the current point and the next point
    point_a_lat = df.iloc[i][lat_col]
    point_a_lon = df.iloc[i][lon_col]
    point_b_lat = df.iloc[i+1][lat_col]
    point_b_lon = df.iloc[i+1][lon_col]

    point_a = (point_a_lat, point_a_lon)
    point_b = (point_b_lat, point_b_lon)

    # Calculate distance in kilometers
    distance = geodesic(point_a, point_b).km

    # Draw the line between the two points
    folium.PolyLine(
        locations=[point_a, point_b],
        color='blue',
        weight=2.5,
        opacity=1,
        tooltip=f"Distance: {distance:.2f} km"
    ).add_to(lithuania_map)

    # Calculate the midpoint for the label position
    midpoint_lat = (point_a_lat + point_b_lat) / 2
    midpoint_lon = (point_a_lon + point_b_lon) / 2

    # Create a custom text label using DivIcon
    folium.Marker(
        location=[midpoint_lat, midpoint_lon],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(75,18), # Center the anchor
            html=f'<div style="font-size: 12pt; color: #2a52be; font-weight: bold; background-color: rgba(255, 255, 255, 0.7); border-radius: 5px; padding: 2px;">{distance:.1f} km</div>',
        )
    ).add_to(lithuania_map)


# --- 5. Save Map ---
lithuania_map.save(output_map_file)

print(f"\nMap successfully created at '{output_map_file}'.")
print("Open this file in a web browser to view it.")


Successfully loaded 52 points from '../data/processed/lithuania_coordinates_cleaned.csv'.

Map successfully created at '../notes/plots/lithuania_map_with_distances.html'.
Open this file in a web browser to view it.


In [7]:
# --- Configuration ---
csv_file_path = '../data/processed/lithuania_coordinates_cleaned.csv'
lat_col = 'latitude'
lon_col = 'longitude'
city_col = 'name'

# The maximum difference in longitude for points to be considered in the same vertical line.
# You can adjust this value: smaller values create tighter, more numerous lines;
# larger values create broader, fewer lines.
LONGITUDE_TOLERANCE = 0.3 

# Map settings
map_center = [55.17, 23.88] 
zoom_start = 7
output_map_file = '../plots/lithuania_meridian_map.html'

# --- 1. Load and Prepare Data ---
try:
    df = pd.read_csv(csv_file_path)
    print(f"Successfully loaded {len(df)} points from '{csv_file_path}'.")
except FileNotFoundError:
    print(f"Error: The file '{csv_file_path}' was not found.")
    exit()

# Sort the DataFrame by longitude to process points from west to east
df.sort_values(by=lon_col, inplace=True)
df.reset_index(drop=True, inplace=True)

# --- 2. Group Points into Vertical Lines ---
# This algorithm groups points that fall within the same vertical "band"
all_lines = []
if not df.empty:
    current_line = [df.iloc[0]] # Start the first line with the first point
    
    for i in range(1, len(df)):
        # Compare the current point's longitude with the first point in the current line
        current_lon = df.iloc[i][lon_col]
        line_start_lon = current_line[0][lon_col]
        
        if abs(current_lon - line_start_lon) < LONGITUDE_TOLERANCE:
            # If it's close enough, add it to the current line
            current_line.append(df.iloc[i])
        else:
            # If it's too far, the vertical line ends. Store it and start a new one.
            all_lines.append(current_line)
            current_line = [df.iloc[i]]
            
    # Add the very last line after the loop finishes
    all_lines.append(current_line)

# --- 3. Filter for Lines with More Than 2 Points ---
valid_lines = [line for line in all_lines if len(line) > 2]
print(f"Found {len(valid_lines)} meridian-like lines with more than 2 points.")

# --- 4. Create the Map ---
lithuania_map = folium.Map(location=map_center, zoom_start=zoom_start)

# Add a marker for every city in the original dataset
for _, row in df.iterrows():
    folium.Marker(
        location=[row[lat_col], row[lon_col]],
        popup=f"{row[city_col]}<br>({row[lat_col]:.4f}, {row[lon_col]:.4f})",
        tooltip=row[city_col]
    ).add_to(lithuania_map)

# Define a list of colors to cycle through for the lines
line_colors = ['blue', 'red', 'green', 'purple', 'orange', 'darkred', 'cadetblue', 'darkgreen', 'darkblue']

# --- 5. Draw the Filtered Lines on the Map ---
for i, line_group in enumerate(valid_lines):
    # Create a list of coordinate pairs for the PolyLine
    line_points = [(point[lat_col], point[lon_col]) for point in line_group]
    
    # Sort points by latitude to draw the line straight from south to north
    line_points.sort(key=lambda x: x[0])
    
    # Get a color for the line, cycling through the list
    color = line_colors[i % len(line_colors)]
    
    # Draw the line
    folium.PolyLine(
        locations=line_points,
        color=color,
        weight=3,
        opacity=0.8,
        tooltip=f"Line Group {i+1} ({len(line_points)} cities)"
    ).add_to(lithuania_map)

# --- 6. Save the Final Map ---
lithuania_map.save(output_map_file)

print(f"\nMap successfully created at '{output_map_file}'.")
print("Open this file in a web browser to view it.")



Successfully loaded 52 points from '../data/processed/lithuania_coordinates_cleaned.csv'.
Found 14 meridian-like lines with more than 2 points.

Map successfully created at '../notes/plots/lithuania_meridian_map.html'.
Open this file in a web browser to view it.


In [8]:
# --- Configuration ---
csv_file_path = '../data/processed/lithuania_coordinates_cleaned.csv'
lat_col = 'latitude'
lon_col = 'longitude'
city_col = 'name'

# Tolerances for grouping points. Adjust these to change sensitivity.
# Smaller values = tighter groups; larger values = broader groups.
LONGITUDE_TOLERANCE = 0.3 # For vertical lines (west-east distance)
LATITUDE_TOLERANCE = 0.2  # For horizontal lines (north-south distance)
MIN_POINTS_PER_LINE = 3   # Minimum number of cities to form a line

# Map settings
map_center = [55.17, 23.88] 
zoom_start = 7
output_map_file = '../plots/lithuania_grid_map.html'

# --- 1. Load Data ---
try:
    df = pd.read_csv(csv_file_path)
    print(f"Successfully loaded {len(df)} points from '{csv_file_path}'.")
except FileNotFoundError:
    print(f"Error: The file '{csv_file_path}' was not found.")
    exit()

# --- 2. Create Base Map ---
lithuania_map = folium.Map(location=map_center, zoom_start=zoom_start)

# Add a marker for every city first, so they are on the bottom layer
for _, row in df.iterrows():
    folium.Marker(
        location=[row[lat_col], row[lon_col]],
        popup=f"{row[city_col]}<br>({row[lat_col]:.4f}, {row[lon_col]:.4f})",
        tooltip=row[city_col]
    ).add_to(lithuania_map)

# --- 3. Generate Vertical (Meridian) Lines ---
df.sort_values(by=lon_col, inplace=True) # Sort west to east
df.reset_index(drop=True, inplace=True)

vertical_lines = []
if not df.empty:
    current_line = [df.iloc[0]]
    for i in range(1, len(df)):
        if abs(df.iloc[i][lon_col] - current_line[0][lon_col]) < LONGITUDE_TOLERANCE:
            current_line.append(df.iloc[i])
        else:
            vertical_lines.append(current_line)
            current_line = [df.iloc[i]]
    vertical_lines.append(current_line)

valid_vertical_lines = [line for line in vertical_lines if len(line) >= MIN_POINTS_PER_LINE]
print(f"Found {len(valid_vertical_lines)} vertical lines with {MIN_POINTS_PER_LINE} or more points.")

# --- 4. Generate Horizontal (Parallel) Lines ---
df.sort_values(by=lat_col, inplace=True) # Sort south to north
df.reset_index(drop=True, inplace=True)

horizontal_lines = []
if not df.empty:
    current_line = [df.iloc[0]]
    for i in range(1, len(df)):
        if abs(df.iloc[i][lat_col] - current_line[0][lat_col]) < LATITUDE_TOLERANCE:
            current_line.append(df.iloc[i])
        else:
            horizontal_lines.append(current_line)
            current_line = [df.iloc[i]]
    horizontal_lines.append(current_line)

valid_horizontal_lines = [line for line in horizontal_lines if len(line) >= MIN_POINTS_PER_LINE]
print(f"Found {len(valid_horizontal_lines)} horizontal lines with {MIN_POINTS_PER_LINE} or more points.")

# --- 5. Draw Vertical Lines on the Map ---
vertical_colors = ['blue', 'darkblue', 'cadetblue']
for i, line_group in enumerate(valid_vertical_lines):
    line_points = [(point[lat_col], point[lon_col]) for point in line_group]
    line_points.sort(key=lambda x: x[0]) # Sort south to north for drawing
    folium.PolyLine(
        locations=line_points,
        color=vertical_colors[i % len(vertical_colors)],
        weight=3,
        opacity=0.7,
        tooltip=f"Vertical Group {i+1} ({len(line_points)} cities)"
    ).add_to(lithuania_map)

# --- 6. Draw Horizontal Lines and Distances on the Map ---
horizontal_colors = ['red', 'darkred', 'orange']
for i, line_group in enumerate(valid_horizontal_lines):
    # Sort group by longitude to draw the line and distances from west to east
    line_group_df = pd.DataFrame(line_group)
    line_group_df.sort_values(by=lon_col, inplace=True)
    
    line_points = list(zip(line_group_df[lat_col], line_group_df[lon_col]))
    
    # Draw the full horizontal line
    folium.PolyLine(
        locations=line_points,
        color=horizontal_colors[i % len(horizontal_colors)],
        weight=3,
        opacity=0.7,
        tooltip=f"Horizontal Group {i+1} ({len(line_points)} cities)"
    ).add_to(lithuania_map)
    
    # Add distance markers for each segment of the line
    for j in range(len(line_points) - 1):
        point_a = line_points[j]
        point_b = line_points[j+1]
        distance = geodesic(point_a, point_b).km
        
        midpoint = ((point_a[0] + point_b[0]) / 2, (point_a[1] + point_b[1]) / 2)
        
        folium.Marker(
            location=midpoint,
            icon=DivIcon(
                icon_size=(150,36),
                icon_anchor=(75,18),
                html=f'<div style="font-size: 11pt; color: #B22222; font-weight: bold; background-color: rgba(255, 255, 255, 0.75); border-radius: 5px; padding: 2px;">{distance:.1f} km</div>',
            )
        ).add_to(lithuania_map)

# --- 7. Save the Final Map ---
lithuania_map.save(output_map_file)

print(f"\nMap successfully created at '{output_map_file}'.")
print("Open this file in a web browser to view it.")


Successfully loaded 52 points from '../data/processed/lithuania_coordinates_cleaned.csv'.
Found 14 vertical lines with 3 or more points.
Found 9 horizontal lines with 3 or more points.

Map successfully created at '../notes/plots/lithuania_grid_map.html'.
Open this file in a web browser to view it.


# LAST ONE

In [10]:
# Map settings
map_center = [55.17, 23.88] 
zoom_start = 7
output_map_file = '../plots/lithuania_shortest_path_map.html'

# --- Helper Function: Traveling Salesperson Solver ---
def find_shortest_path(points_df):
    """
    Finds the shortest path connecting all points in a DataFrame.
    Returns the optimal order of indices and the total distance.
    """
    if len(points_df) < 2:
        return points_df.index.tolist(), 0

    indices = points_df.index.tolist()
    min_path_indices = None
    min_distance = float('inf')
    
    start_index = indices[0]
    other_indices = indices[1:]

    for perm_indices in permutations(other_indices):
        current_path_indices = [start_index] + list(perm_indices)
        current_distance = 0
        
        for i in range(len(current_path_indices) - 1):
            point_a_idx = current_path_indices[i]
            point_b_idx = current_path_indices[i+1]
            point_a_coords = (points_df.loc[point_a_idx, lat_col], points_df.loc[point_a_idx, lon_col])
            point_b_coords = (points_df.loc[point_b_idx, lat_col], points_df.loc[point_b_idx, lon_col])
            current_distance += geodesic(point_a_coords, point_b_coords).km
        
        if current_distance < min_distance:
            min_distance = current_distance
            min_path_indices = current_path_indices
            
    return min_path_indices, min_distance

# --- 1. Load Data ---
try:
    df = pd.read_csv(csv_file_path)
    print(f"Successfully loaded {len(df)} points from '{csv_file_path}'.")
except FileNotFoundError:
    print(f"Error: The file '{csv_file_path}' was not found.")
    exit()

# --- 2. Create Base Map ---
lithuania_map = folium.Map(location=map_center, zoom_start=zoom_start)
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row[lat_col], row[lon_col]], radius=3, color='gray', fill=True,
        fill_color='gray', fill_opacity=0.6, popup=row[city_col]
    ).add_to(lithuania_map)

# --- 3. Group Vertical and Horizontal Lines ---
# (This logic remains the same as before)
df.sort_values(by=lon_col, inplace=True)
vertical_lines_groups = []
if not df.empty:
    current_line = [df.iloc[0].to_dict()]
    for i in range(1, len(df)):
        if abs(df.iloc[i][lon_col] - current_line[0][lon_col]) < LONGITUDE_TOLERANCE:
            current_line.append(df.iloc[i].to_dict())
        else:
            vertical_lines_groups.append(pd.DataFrame(current_line))
            current_line = [df.iloc[i].to_dict()]
    vertical_lines_groups.append(pd.DataFrame(current_line))
valid_vertical_lines = [g for g in vertical_lines_groups if len(g) >= MIN_POINTS_PER_LINE]
print(f"Found {len(valid_vertical_lines)} vertical lines.")

df.sort_values(by=lat_col, inplace=True)
horizontal_lines_groups = []
if not df.empty:
    current_line = [df.iloc[0].to_dict()]
    for i in range(1, len(df)):
        if abs(df.iloc[i][lat_col] - current_line[0][lat_col]) < LATITUDE_TOLERANCE:
            current_line.append(df.iloc[i].to_dict())
        else:
            horizontal_lines_groups.append(pd.DataFrame(current_line))
            current_line = [df.iloc[i].to_dict()]
    horizontal_lines_groups.append(pd.DataFrame(current_line))
valid_horizontal_lines = [g for g in horizontal_lines_groups if len(g) >= MIN_POINTS_PER_LINE]
print(f"Found {len(valid_horizontal_lines)} horizontal lines.")

# --- 4. Process Paths, Draw on Map, and Collect Data for Export ---
all_path_data = []
all_line_groups = ([(g, 'Vertical') for g in valid_vertical_lines] + 
                   [(g, 'Horizontal') for g in valid_horizontal_lines])
line_colors = ['#377eb8', '#e41a1c', '#4daf4a', '#984ea3', '#ff7f00', '#ffff33', '#a65628']

for i, (group_df, group_type) in enumerate(all_line_groups):
    group_df.reset_index(drop=True, inplace=True)
    
    # Find the shortest path for the current group
    optimal_indices, total_distance = find_shortest_path(group_df)
    
    # Reorder the DataFrame according to the optimal path
    ordered_df = group_df.loc[optimal_indices].reset_index(drop=True)
    
    # Get coordinates and city names in the optimal order
    shortest_path_coords = list(zip(ordered_df[lat_col], ordered_df[lon_col]))
    road_path_names = " -> ".join(ordered_df[city_col])
    
    # Calculate segment distances and create a detailed string
    segment_details = []
    for j in range(len(ordered_df) - 1):
        city_a = ordered_df.loc[j, city_col]
        city_b = ordered_df.loc[j+1, city_col]
        dist = geodesic(shortest_path_coords[j], shortest_path_coords[j+1]).km
        segment_details.append(f"{city_a} -> {city_b}: {dist:.2f} km")
    
    # Store all collected data for this path
    all_path_data.append({
        'group_id': f'Group {i+1} ({group_type})',
        'road_path': road_path_names,
        'total_distance_km': round(total_distance, 2),
        'segment_distances': "; ".join(segment_details)
    })
    
    # --- Drawing on the map (same as before) ---
    color = line_colors[i % len(line_colors)]
    folium.PolyLine(locations=shortest_path_coords, color=color, weight=3, opacity=0.8,
                    tooltip=f"Group {i+1} ({group_type})").add_to(lithuania_map)
    
    for j in range(len(shortest_path_coords) - 1):
        point_a, point_b = shortest_path_coords[j], shortest_path_coords[j+1]
        dist = geodesic(point_a, point_b).km
        midpoint = ((point_a[0] + point_b[0]) / 2, (point_a[1] + point_b[1]) / 2)
        folium.Marker(
            location=midpoint,
            icon=DivIcon(
                icon_size=(150,36), icon_anchor=(75,18),
                html=f'<div style="font-size: 11pt; color: {color}; font-weight: bold; background-color: rgba(255, 255, 255, 0.75); border-radius: 5px; padding: 2px;">{dist:.1f} km</div>',
            )
        ).add_to(lithuania_map)

# --- 5. Save Data to CSV and Map to HTML ---
# Save the collected path data
if all_path_data:
    results_df = pd.DataFrame(all_path_data)
    results_df.to_csv(output_csv_file, index=False, encoding='utf-8')
    print(f"\nSuccessfully saved road data to '{output_csv_file}'.")
else:
    print("\nNo valid paths were found to save.")

# Save the map
lithuania_map.save(output_map_file)
print(f"Map successfully created at '{output_map_file}'.")



Successfully loaded 52 points from '../data/processed/lithuania_coordinates_cleaned.csv'.
Found 14 vertical lines.
Found 9 horizontal lines.

Successfully saved road data to '../notes/lithuanian_road_distances.csv'.
Map successfully created at '../notes/plots/lithuania_shortest_path_map.html'.
